In [1]:
playersPath = "I:/FM/Football Manager 2023/graphics/1999/"
shortlist = "I:/FM/Football Manager 2023/Shortlist.html"
rename = True

In [2]:
import os, os.path

In [3]:
from lxml import etree as ET

In [4]:
import pandas as pd

In [5]:
newgens = pd.read_html(shortlist,encoding="UTF-8")[0]

In [6]:
newgens = newgens.drop(["Rec","Inf"], axis=1)


In [7]:
print(newgens)

             UID               Name
0     2002116244  Tommaso Abbrescia
1     2002144788      Miguel Aceval
2     2002144802     Albert Acevedo
3     2002152485  Emmanuel Adebayor
4     2002107683         Adem Koçak
...          ...                ...
1111  2002115579      Daniel Ziebig
1112  2002110370      François Zoko
1113  2002152859        Ramon Zomer
1114  2002144492               Zulu
1115  2002112979     Theo Zwarthoed

[1116 rows x 2 columns]


In [8]:
newgens = newgens[~newgens["Name"].duplicated(keep=False)]

In [9]:
print(newgens)

             UID               Name
0     2002116244  Tommaso Abbrescia
1     2002144788      Miguel Aceval
2     2002144802     Albert Acevedo
3     2002152485  Emmanuel Adebayor
4     2002107683         Adem Koçak
...          ...                ...
1111  2002115579      Daniel Ziebig
1112  2002110370      François Zoko
1113  2002152859        Ramon Zomer
1114  2002144492               Zulu
1115  2002112979     Theo Zwarthoed

[1102 rows x 2 columns]


In [10]:
newgens["Deaccented"] = newgens["Name"].str.normalize("NFKD").str.encode("ascii",errors="ignore").str.decode("utf-8")

In [11]:
files = os.listdir(playersPath)

In [13]:
names = [os.path.splitext(file)[0] for file in files]

In [14]:
print(f"Players found: {len(names)} : {names}")

Players found: 434 : ['2002105845', '2002105846', '2002105847', '2002105877', '2002105878', '2002105908', '2002105931', '2002105990', '2002106465', '2002106521', '2002106764', '2002106766', '2002106767', '2002106768', '2002106810', '2002106814', '2002106815', '2002107046', '2002107095', '2002107141', '2002107247', '2002107267', '2002107331', '2002107428', '2002107445', '2002107446', '2002107450', '2002107594', '2002107656', '2002107662', '2002107670', '2002107681', '2002107682', '2002107683', '2002107700', '2002107719', '2002107809', '2002107812', '2002107823', '2002107824', '2002107974', '2002107979', '2002107981', '2002107988', '2002108057', '2002108058', '2002108061', '2002108214', '2002108215', '2002108361', '2002108362', '2002108367', '2002108371', '2002108372', '2002108373', '2002108376', '2002108385', '2002108386', '2002108387', '2002108389', '2002108395', '2002108424', '2002108425', '2002108627', '2002108633', '2002108643', '2002108644', '2002108645', '2002108647', '2002108659'

In [15]:
Players = newgens.loc[newgens["UID"].astype(str).isin(names) | newgens["Name"].isin(names) | newgens["Deaccented"].isin(names)]


In [16]:
config = ET.Element("record")
preload = ET.SubElement(config,"boolean", id="preload", value="false")
amap = ET.SubElement(config,"boolean", id="amap", value="false")
list = ET.SubElement(config, "list", id="maps")


In [17]:
for index,player in Players.iterrows():
    record = {"from":None, "to":None}
    
    if(os.path.exists(f"{playersPath}{player['Name']}.png")):
        record["to"] = player["UID"]
        if rename: 
            os.rename(f"{playersPath}{player['Name']}.png", f"{playersPath}{player['UID']}.png")
            record["from"] = player['UID']
        else:
            record["from"] = player["Name"]
    elif(os.path.exists(f"{playersPath}{player['Deaccented']}.png")):
        record["to"] = player["UID"]
        if rename: 
            os.rename(f"{playersPath}{player['Deaccented']}.png", f"{playersPath}{player['UID']}.png")
            record["from"] = player['UID']
        else:
            record["from"] = player["Deaccented"]
    elif(os.path.exists(f"{playersPath}{player['UID']}.png")):
        record["to"] = player["UID"]
        record["from"] = player['UID']
        
    if record:
        recordXML = ET.SubElement(list, "record")
        recordXML.set("from", str(record["from"]))
        recordXML.set("to",f'graphics/pictures/person/{record["to"]}/portrait')
    
        

In [18]:

configTree = config.getroottree()
configTree.write(f"{playersPath}/config.xml", encoding="utf-8", pretty_print=True)